In [ ]:
import pandas as pd
import numpy as np
import pickle as pk
import seaborn as sns
from collections import deque
import os
import gc
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [ ]:
from asset import Unit_dict, Upgrade_dict, Building_dict, Research_dict

In [ ]:
%%time
# data load
f = open('./data/train.bin', 'rb')
tr = pk.load(f)
f.close
f = open('./data/test.bin', 'rb')
te = pk.load(f)
f.close()
sub = pd.read_csv('./data/sample_submission.csv')

tr['event_contents'] = tr['event_contents'].replace(np.nan, 'NA')
te['event_contents'] = te['event_contents'].replace(np.nan, 'NA')

target = tr.groupby('game_id').mean()['winner']


In [ ]:
f = open('./data/sub/tr_u.bin', 'rb')
tr_u = pk.load(f)
f.close()
f = open('./data/sub/tr_b.bin', 'rb')
tr_b = pk.load(f)
f.close()
f = open('./data/sub/tr_re.bin', 'rb')
tr_re = pk.load(f)
f.close()
f = open('./data/sub/tr_up.bin', 'rb')
tr_up = pk.load(f)
f.close()

f = open('./data/sub/te_u.bin', 'rb')
te_u = pk.load(f)
f.close()
f = open('./data/sub/te_b.bin', 'rb')
te_b = pk.load(f)
f.close()
f = open('./data/sub/te_re.bin', 'rb')
te_re = pk.load(f)
f.close()
f = open('./data/sub/te_up.bin', 'rb')
te_up = pk.load(f)
f.close()

In [ ]:
tr_n = np.unique(tr['game_id'])
te_n = np.unique(te['game_id'])
temp_tr = tr[tr['event'] == 0]
temp_te = te[te['event'] == 0]

del tr, te

# 1. Unit

In [ ]:
test = np.zeros((len(tr_u), 11))
test0 = test.copy()
test1 = test.copy()
test2 = test.copy()

test01 = test.copy()
test11 = test.copy()
test21 = test.copy()

for i in tqdm(range(len(temp_tr))):
    for k in Unit_dict.keys():
        if k in temp_tr['event_contents'].values[i]:
            n = temp_tr['time'].values[i] // 60
            if temp_tr['player'].values[i] == 0:
                test0[temp_tr['game_id'].values[i]][n] += Unit_dict[k][0]
                test1[temp_tr['game_id'].values[i]][n] += Unit_dict[k][1]
                test2[temp_tr['game_id'].values[i]][n] += Unit_dict[k][2]
            else:
                test01[temp_tr['game_id'].values[i]][n] += Unit_dict[k][0]
                test11[temp_tr['game_id'].values[i]][n] += Unit_dict[k][1]
                test21[temp_tr['game_id'].values[i]][n] += Unit_dict[k][2]

cols = ['u_mine_m_0', 'u_gas_m_0', 'u_0', 'u_mine_m_1', 'u_gas_m_1', 'u_1', 'u_mine_m', 'u_gas_m', 'u_m']
new = []
for c in cols.copy():
    for i in range(11):
        new.append(c + '_t' + str(i))

new_cols = list(tr_u.columns) + new

test02 = test0 - test01
test12 = test1 - test11
test22 = test2 - test21
k = np.concatenate([tr_u.values, test0, test1, test2, test01, test11, test21, test02, test12, test22], axis=1)

tr_u_concat = pd.DataFrame(k, columns=new_cols)



In [ ]:
test = np.zeros((len(te_u), 11))
test0 = test.copy()
test1 = test.copy()
test2 = test.copy()

test01 = test.copy()
test11 = test.copy()
test21 = test.copy()

for i in tqdm(range(len(temp_te))):
    for k in Unit_dict.keys():
        if k in temp_te['event_contents'].values[i]:
            n = temp_te['time'].values[i] // 60
            if temp_te['player'].values[i] == 0:
                test0[temp_te['game_id'].values[i]-38872][n] += Unit_dict[k][0]
                test1[temp_te['game_id'].values[i]-38872][n] += Unit_dict[k][1]
                test2[temp_te['game_id'].values[i]-38872][n] += Unit_dict[k][2]
            else:
                test01[temp_te['game_id'].values[i]-38872][n] += Unit_dict[k][0]
                test11[temp_te['game_id'].values[i]-38872][n] += Unit_dict[k][1]
                test21[temp_te['game_id'].values[i]-38872][n] += Unit_dict[k][2]

cols = ['u_mine_m_0', 'u_gas_m_0', 'u_0', 'u_mine_m_1', 'u_gas_m_1', 'u_1', 'u_mine_m', 'u_gas_m', 'u_m']
new = []
for c in cols.copy():
    for i in range(11):
        new.append(c + '_t' + str(i))

new_cols = list(te_u.columns) + new

test02 = test0 - test01
test12 = test1 - test11
test22 = test2 - test21
k = np.concatenate([te_u.values, test0, test1, test2, test01, test11, test21, test02, test12, test22], axis=1)

te_u_concat = pd.DataFrame(k, columns=new_cols)



# 2. Building

In [ ]:
test = np.zeros((len(tr_b), 11))
test0 = test.copy()
test1 = test.copy()
test2 = test.copy()

test01 = test.copy()
test11 = test.copy()
test21 = test.copy()

for i in tqdm(range(len(temp_tr))):
    for k in Building_dict.keys():
        if k in temp_tr['event_contents'].values[i]:
            n = temp_tr['time'].values[i] // 60
            if temp_tr['player'].values[i] == 0:
                test0[temp_tr['game_id'].values[i]][n] += Building_dict[k][0]
                test1[temp_tr['game_id'].values[i]][n] += Building_dict[k][1]
                test2[temp_tr['game_id'].values[i]][n] += 1
            else:
                test01[temp_tr['game_id'].values[i]][n] += Building_dict[k][0]
                test11[temp_tr['game_id'].values[i]][n] += Building_dict[k][1]
                test21[temp_tr['game_id'].values[i]][n] += 1

cols = ['b_mine_m_0', 'b_gas_m_0', 'b_0', 'b_mine_m_1', 'b_gas_m_1', 'b_1', 'b_mine_m', 'b_gas_m', 'b_m']
new = []
for c in cols.copy():
    for i in range(11):
        new.append(c + '_t' + str(i))

new_cols = list(tr_b.columns) + new

test02 = test0 - test01
test12 = test1 - test11
test22 = test2 - test21
k = np.concatenate([tr_b.values, test0, test1, test2, test01, test11, test21, test02, test12, test22], axis=1)

pd.DataFrame(k, columns=new_cols)

tr_b_concat = pd.DataFrame(k, columns=new_cols)


In [ ]:
test = np.zeros((len(te_b), 11))
test0 = test.copy()
test1 = test.copy()
test2 = test.copy()

test01 = test.copy()
test11 = test.copy()
test21 = test.copy()

for i in tqdm(range(len(temp_te))):
    for k in Building_dict.keys():
        if k in temp_te['event_contents'].values[i]:
            n = temp_te['time'].values[i] // 60
            if temp_te['player'].values[i] == 0:
                test0[temp_te['game_id'].values[i]-38872][n] += Building_dict[k][0]
                test1[temp_te['game_id'].values[i]-38872][n] += Building_dict[k][1]
                test2[temp_te['game_id'].values[i]-38872][n] += 1
            else:
                test01[temp_te['game_id'].values[i]-38872][n] += Building_dict[k][0]
                test11[temp_te['game_id'].values[i]-38872][n] += Building_dict[k][1]
                test21[temp_te['game_id'].values[i]-38872][n] += 1

cols = ['b_mine_m_0', 'b_gas_m_0', 'b_0', 'b_mine_m_1', 'b_gas_m_1', 'b_1', 'b_mine_m', 'b_gas_m', 'b_m']
new = []
for c in cols.copy():
    for i in range(11):
        new.append(c + '_t' + str(i))

new_cols = list(te_b.columns) + new

test02 = test0 - test01
test12 = test1 - test11
test22 = test2 - test21
k = np.concatenate([te_b.values, test0, test1, test2, test01, test11, test21, test02, test12, test22], axis=1)

pd.DataFrame(k, columns=new_cols)

te_b_concat = pd.DataFrame(k, columns=new_cols)


# 3. Upgrade

In [ ]:
test = np.zeros((len(tr_up), 11))
test0 = test.copy()
test1 = test.copy()
test2 = test.copy()

test01 = test.copy()
test11 = test.copy()
test21 = test.copy()

for i in tqdm(range(len(temp_tr))):
    for k in Upgrade_dict.keys():
        if k in temp_tr['event_contents'].values[i]:
            n = temp_tr['time'].values[i] // 60
            if temp_tr['player'].values[i] == 0:
                test0[temp_tr['game_id'].values[i]][n] += Upgrade_dict[k][0]
                test1[temp_tr['game_id'].values[i]][n] += Upgrade_dict[k][1]
                test2[temp_tr['game_id'].values[i]][n] += 1
            else:
                test01[temp_tr['game_id'].values[i]][n] += Upgrade_dict[k][0]
                test11[temp_tr['game_id'].values[i]][n] += Upgrade_dict[k][1]
                test21[temp_tr['game_id'].values[i]][n] += 1

cols = ['up_mine_m_0', 'up_gas_m_0', 'up_0', 'up_mine_m_1', 'up_gas_m_1', 'up_1', 'up_mine_m', 'up_gas_m', 'up_m']
new = []
for c in cols.copy():
    for i in range(11):
        new.append(c + '_t' + str(i))

new_cols = list(tr_up.columns) + new

test02 = test0 - test01
test12 = test1 - test11
test22 = test2 - test21
k = np.concatenate([tr_up.values, test0, test1, test2, test01, test11, test21, test02, test12, test22], axis=1)

pd.DataFrame(k, columns=new_cols)

tr_up_concat = pd.DataFrame(k, columns=new_cols)


In [ ]:
test = np.zeros((len(te_u), 11))
test0 = test.copy()
test1 = test.copy()
test2 = test.copy()

test01 = test.copy()
test11 = test.copy()
test21 = test.copy()

for i in tqdm(range(len(temp_te))):
    for k in Upgrade_dict.keys():
        if k in temp_te['event_contents'].values[i]:
            n = temp_te['time'].values[i] // 60
            if temp_te['player'].values[i] == 0:
                test0[temp_te['game_id'].values[i]-38872][n] += Upgrade_dict[k][0]
                test1[temp_te['game_id'].values[i]-38872][n] += Upgrade_dict[k][1]
                test2[temp_te['game_id'].values[i]-38872][n] += 1
            else:
                test01[temp_te['game_id'].values[i]-38872][n] += Upgrade_dict[k][0]
                test11[temp_te['game_id'].values[i]-38872][n] += Upgrade_dict[k][1]
                test21[temp_te['game_id'].values[i]-38872][n] += 1

cols = ['up_mine_m_0', 'up_gas_m_0', 'up_0', 'up_mine_m_1', 'up_gas_m_1', 'up_1', 'up_mine_m', 'up_gas_m', 'up_m']
new = []
for c in cols.copy():
    for i in range(11):
        new.append(c + '_t' + str(i))

new_cols = list(te_up.columns) + new

test02 = test0 - test01
test12 = test1 - test11
test22 = test2 - test21
k = np.concatenate([te_up.values, test0, test1, test2, test01, test11, test21, test02, test12, test22], axis=1)

pd.DataFrame(k, columns=new_cols)

te_up_concat = pd.DataFrame(k, columns=new_cols)


# 4. Research

In [ ]:
test = np.zeros((len(tr_re), 11))
test0 = test.copy()
test1 = test.copy()
test2 = test.copy()

test01 = test.copy()
test11 = test.copy()
test21 = test.copy()

for i in tqdm(range(len(temp_tr))):
    for k in Research_dict.keys():
        if k in temp_tr['event_contents'].values[i]:
            n = temp_tr['time'].values[i] // 60
            if temp_tr['player'].values[i] == 0:
                test0[temp_tr['game_id'].values[i]][n] += Research_dict[k][0]
                test1[temp_tr['game_id'].values[i]][n] += Research_dict[k][1]
                test2[temp_tr['game_id'].values[i]][n] += 1
            else:
                test01[temp_tr['game_id'].values[i]][n] += Research_dict[k][0]
                test11[temp_tr['game_id'].values[i]][n] += Research_dict[k][1]
                test21[temp_tr['game_id'].values[i]][n] += 1

cols = ['re_mine_m_0', 're_gas_m_0', 're_0', 're_mine_m_1', 're_gas_m_1', 're_1', 're_mine_m', 're_gas_m', 're_m']
new = []
for c in cols.copy():
    for i in range(11):
        new.append(c + '_t' + str(i))

new_cols = list(tr_re.columns) + new

test02 = test0 - test01
test12 = test1 - test11
test22 = test2 - test21
k = np.concatenate([tr_re.values, test0, test1, test2, test01, test11, test21, test02, test12, test22], axis=1)

pd.DataFrame(k, columns=new_cols)

tr_re_concat = pd.DataFrame(k, columns=new_cols)


In [ ]:
test = np.zeros((len(te_re), 11))
test0 = test.copy()
test1 = test.copy()
test2 = test.copy()

test01 = test.copy()
test11 = test.copy()
test21 = test.copy()

for i in tqdm(range(len(temp_te))):
    for k in Research_dict.keys():
        if k in temp_te['event_contents'].values[i]:
            n = temp_te['time'].values[i] // 60
            if temp_te['player'].values[i] == 0:
                test0[temp_te['game_id'].values[i]-38872][n] += Research_dict[k][0]
                test1[temp_te['game_id'].values[i]-38872][n] += Research_dict[k][1]
                test2[temp_te['game_id'].values[i]-38872][n] += 1
            else:
                test01[temp_te['game_id'].values[i]-38872][n] += Research_dict[k][0]
                test11[temp_te['game_id'].values[i]-38872][n] += Research_dict[k][1]
                test21[temp_te['game_id'].values[i]-38872][n] += 1

cols = ['re_mine_m_0', 're_gas_m_0', 're_0', 're_mine_m_1', 're_gas_m_1', 're_1', 're_mine_m', 're_gas_m', 're_m']
new = []
for c in cols.copy():
    for i in range(11):
        new.append(c + '_t' + str(i))

new_cols = list(te_re.columns) + new

test02 = test0 - test01
test12 = test1 - test11
test22 = test2 - test21
k = np.concatenate([te_re.values, test0, test1, test2, test01, test11, test21, test02, test12, test22], axis=1)

pd.DataFrame(k, columns=new_cols)

te_re_concat = pd.DataFrame(k, columns=new_cols)


In [ ]:
# save

In [ ]:
f = open('./data/sub/tr_u_concat.bin', 'wb')
pk.dump(tr_u_concat, f)
f.close()
f = open('./data/sub/tr_b_concat.bin', 'wb')
pk.dump(tr_b_concat, f)
f.close()
f = open('./data/sub/tr_up_concat.bin', 'wb')
pk.dump(tr_up_concat, f)
f.close()
f = open('./data/sub/tr_re_concat.bin', 'wb')
pk.dump(tr_re_concat, f)
f.close()

f = open('./data/sub/te_u_concat.bin', 'wb')
pk.dump(te_u_concat, f)
f.close()
f = open('./data/sub/te_b_concat.bin', 'wb')
pk.dump(te_b_concat, f)
f.close()
f = open('./data/sub/te_up_concat.bin', 'wb')
pk.dump(te_up_concat, f)
f.close()
f = open('./data/sub/te_re_concat.bin', 'wb')
pk.dump(te_re_concat, f)
f.close()